<h1>Generative Album Art Adversarial Network (GAAAN)</h1>
This dcgan is an album art generator, adapted from the following articles
https://towardsdatascience.com/generating-modern-arts-using-generative-adversarial-network-gan-on-spell-39f67f83c7b4 

https://colab.research.google.com/drive/1Mxbfn0BUW4BlgEPc-minaE_M0_PaYIIX#scrollTo=6AIl-8kWThY8

Read more about this project here: https://github.com/mmcdermott011/GAAAN

In [0]:
from google.colab import drive
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


In [0]:
%cd "/content/drive/My Drive/Deep_Learning_Project"
!ls


/content/drive/My Drive/Deep_Learning_Project
logs	       metal_resized.zip  resized_imgs	    saved_models
metal_resized  output		  resized_imgs.zip


In [0]:
import tensorflow as tf
import numpy as np
from matplotlib import pyplot as plt, gridspec
import os
import imageio

# Generation resolution - Must be square
# Training data is also scaled to this.
# Note GENERATE_RES 4 or higher  will blow Google CoLab's memory and have not
# been tested extensively.
from PIL import Image
from keras import Sequential, Input, Model
from keras.layers import Dense, Conv2D, LeakyReLU, Dropout, ZeroPadding2D, BatchNormalization, Flatten, Activation, \
    UpSampling2D, Reshape
from keras.optimizers import Adam
from keras.utils import plot_model
from keras.utils import model_to_dot
import glob
from keras import backend as K
from keras.utils import model_to_dot
from keras.models import load_model

# Preview image
PREVIEW_ROWS = 4
PREVIEW_COLS = 4
PREVIEW_MARGIN = 16
SAVE_FREQ = 500
# Size vector to generate images from
NOISE_SIZE = 100
# Size vector to generate images from
SEED_SIZE = 100

# Configuration
IMAGE_DIR = 'metal_resized/*.jp*'
images_path = IMAGE_DIR
LOG_DIR = 'logs/'
SAVE_MODEL_DIR = 'saved_models/'

BUFFER_SIZE = 60000

# Configuration
EPOCHS = 50000 # number of iterations
BATCH_SIZE = 64
GENERATE_RES = 3 # Generation resolution factor (1=32, 2=64, 3=96, 4=128, etc.)
IMAGE_SIZE = 128 # rows/cols
IMAGE_CHANNELS = 3

GENERATE_SQUARE = 128 # rows/cols (should be square)

disc_model = 'disc.hdf5' 
gen_model = 'gen.hdf5'



Using TensorFlow backend.


In [0]:
# A function to normalize image pixels.
def norm_img(img):
    '''A function to Normalize Images.
    Input:
        img : Original image as numpy array.
    Output: Normailized Image as numpy array
    '''
    img = (img / 127.5) - 1
    return img

def denorm_img(img):
    '''A function to Denormailze, i.e. recreate image from normalized image
    Input:
        img : Normalized image as numpy array.
    Output: Original Image as numpy array
    '''
    img = (img + 1) * 127.5
    return img.astype(np.uint8)

def sample_from_dataset(batch_size, data_dir=None):
    '''Create a batch of image samples by sampling random images from a data directory.
    Assumes images are already the correct size, and will normalize the images.
    Input:
        batch_size : Sample size required
        data_dir : Path of directory where training images are placed.

    Output:
        sample : batch of processed images
    '''
    sample_dim = (batch_size,) + image_shape
    sample = np.empty(sample_dim, dtype=np.float32)
    all_data_dirlist = list(glob.glob(data_dir))
    sample_imgs_paths = np.random.choice(all_data_dirlist,batch_size)
    for index,img_filename in enumerate(sample_imgs_paths):
        image = Image.open(img_filename)
        image = image.resize(image_shape[:-1])
        image = image.convert('RGB')
        image = np.asarray(image)
        image = norm_img(image)
        sample[index,...] = image
    return sample

def gen_noise(batch_size, noise_shape):
    ''' Generates a numpy vector sampled from normal distribution of shape (batch_size,noise_shape)
    Input:
        batch_size : size of batch
        noise_shape: shape of noise vector, normally kept as 100
    Output:a numpy vector sampled from normal distribution of shape (batch_size,noise_shape)
    '''
    return np.random.normal(0, 1, size=(batch_size,)+noise_shape)



def build_discriminator(image_shape):
    model = Sequential()
    model.add(Conv2D(32, kernel_size=4, strides=2, input_shape=image_shape, padding="same"))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.25))
    model.add(Conv2D(64, kernel_size=4, strides=2, padding="same"))
    model.add(ZeroPadding2D(padding=((0, 1), (0, 1))))
    model.add(BatchNormalization(momentum=0.8))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.25))
    model.add(Conv2D(128, kernel_size=4, strides=2, padding="same"))
    model.add(BatchNormalization(momentum=0.8))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.25))
    model.add(Conv2D(256, kernel_size=4, strides=2, padding="same"))
    model.add(BatchNormalization(momentum=0.8))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Conv2D(512, kernel_size=4, strides=2, padding="same"))
    model.add(BatchNormalization(momentum=0.8))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(1, activation='sigmoid'))
    dis_opt = Adam(lr=0.0015, beta_1=0.5)

    input_image = Input(shape=image_shape)
    validity = model(input_image)
    discriminator_model = Model(input = input_image, output = validity)
    discriminator_model.compile(loss='binary_crossentropy', optimizer=dis_opt, metrics=['accuracy'])
    discriminator_model.summary()
    return discriminator_model


def build_generator(noise_size, channels):
    model = Sequential()
    model.add(Dense(8 * 8 * 512, activation="relu", input_dim = noise_size))
    model.add(Reshape((8, 8, 512)))
    model.add(UpSampling2D())

    model.add(Conv2D(512, kernel_size=4, strides = 1, padding="same"))
    model.add(BatchNormalization(momentum=0.8))
    model.add(LeakyReLU(alpha=0.2))

    model.add(UpSampling2D())
    model.add(Conv2D(256, kernel_size=4,strides = 1, padding="same"))
    model.add(BatchNormalization(momentum=0.8))
    model.add(LeakyReLU(alpha=0.2))

    model.add(UpSampling2D())
    model.add(Conv2D(128, kernel_size=4, strides =1, padding="same"))
    model.add(BatchNormalization(momentum=0.8))
    model.add(LeakyReLU(alpha=0.2))

    model.add(UpSampling2D())
    model.add(Conv2D(channels, kernel_size=4, strides = 1, padding="same"))
    model.add(Activation("tanh"))
    model.summary()
    gen_input = Input(shape=(noise_size,))
    generated_image = model(gen_input)
    gen_opt = Adam(lr=0.003, beta_1=0.5)
    generator = Model(input = gen_input, output = generated_image)
    generator.compile(loss='binary_crossentropy', optimizer=gen_opt, metrics=['accuracy'])
    return generator


def save_images(cnt, noise):
    image_array = np.full((
        PREVIEW_MARGIN + (PREVIEW_ROWS * (IMAGE_SIZE + PREVIEW_MARGIN)),
        PREVIEW_MARGIN + (PREVIEW_COLS * (IMAGE_SIZE + PREVIEW_MARGIN)), 3),
        255, dtype=np.uint8)
    generated_images = generator.predict(noise)
    generated_images = 0.5 * generated_images + 0.5
    image_count = 0
    for row in range(PREVIEW_ROWS):
        for col in range(PREVIEW_COLS):
            r = row * (IMAGE_SIZE + PREVIEW_MARGIN) + PREVIEW_MARGIN
            c = col * (IMAGE_SIZE + PREVIEW_MARGIN) + PREVIEW_MARGIN
            image_array[r:r + IMAGE_SIZE, c:c +
                        IMAGE_SIZE] = generated_images[image_count] * 255
            image_count += 1
    output_path = 'output'
    if not os.path.exists(output_path):
        os.makedirs(output_path)
    filename = os.path.join(output_path, f"trained-{cnt}.png")
    im = Image.fromarray(image_array)
    im.save(filename)



def save_img_batch(img_batch,img_save_dir):
    '''Takes as input a image batch and a img_save_dir and saves 16 images from the batch in a 4x4 grid in the img_save_dir
    '''
    plt.figure(figsize=(16,16))
    gs1 = gridspec.GridSpec(4, 4)
    gs1.update(wspace=0, hspace=0)
    rand_indices = np.random.choice(img_batch.shape[0],16,replace=False)
    for i in range(16):
        ax1 = plt.subplot(gs1[i])
        ax1.set_aspect('equal')
        rand_index = rand_indices[i]
        image = img_batch[rand_index, :,:,:]
        fig = plt.imshow(denorm_img(image))
        plt.axis('off')
        fig.axes.get_xaxis().set_visible(False)
        fig.axes.get_yaxis().set_visible(False)
    plt.tight_layout()
    plt.savefig(img_save_dir,bbox_inches='tight',pad_inches=0)
    plt.show()

import imageio
img_save_dir = 'output/'
generated_images = [img_save_dir+'trained-'+str(x)+".png" for x in range(1,74)]
def make_gif():
    images = []
    for filename in generated_images:
        images.append(imageio.imread(filename))

    imageio.mimsave(img_save_dir + 'metal_v3.5.gif', images)

#make_gif()

In [0]:
image_shape = (IMAGE_SIZE, IMAGE_SIZE, IMAGE_CHANNELS)

#iscriminator = load_model(disc_model)
#generator = load_model(gen_model)

discriminator = build_discriminator(image_shape)
generator = build_generator(NOISE_SIZE, IMAGE_CHANNELS)

discriminator.trainable = False

optimizer = Adam(1.5e-3, 0.5)

gen_inp = Input(shape=(NOISE_SIZE,))
GAN_inp = generator(gen_inp)
GAN_opt = discriminator(GAN_inp)

gan = Model(gen_inp, GAN_opt)
gan.compile(loss="binary_crossentropy",optimizer = optimizer, metrics = ["accuracy"])
gan.summary()


fixed_noise = np.random.normal(0, 1, (PREVIEW_ROWS * PREVIEW_COLS, NOISE_SIZE))


#plot_model(generator, to_file='generator.png')
#plot_model(discriminator, to_file='discriminator.png')
avg_disc_fake_loss = []
avg_disc_real_loss = []
avg_GAN_loss = []

cnt = 1
for epoch in range(EPOCHS):
    if(epoch%10 == 0):
      print("EPOCH: " + str(epoch))
    #idx = np.random.randint(0, training_data.shape[0], BATCH_SIZE)
    #x_real = training_data[idx]
    x_real = sample_from_dataset(BATCH_SIZE, data_dir=IMAGE_DIR)


    noise = np.random.normal(0, 1, (BATCH_SIZE, NOISE_SIZE))
    x_fake = generator.predict(noise)

    y_real = np.ones((BATCH_SIZE, 1)) - np.random.random_sample(BATCH_SIZE) * 0.2
    y_fake = np.random.random_sample(BATCH_SIZE) * 0.2

    discriminator.trainable = True
    generator.trainable = False

    discriminator_metric_real = discriminator.train_on_batch(x_real, y_real[0])
    discriminator_metric_generated = discriminator.train_on_batch(x_fake, y_fake)

    avg_disc_fake_loss.append(discriminator_metric_real[0])
    avg_disc_real_loss.append(discriminator_metric_generated[0])

    generator.trainable = True
    discriminator.trainable = False

    discriminator_metric = 0.5 * np.add(discriminator_metric_real, discriminator_metric_generated)
    generator_metric = gan.train_on_batch(noise, y_real[0])
    generator_metric = gan.train_on_batch(noise, y_real[0])
    avg_GAN_loss.append(generator_metric[0])


    if epoch % SAVE_FREQ == 0:
        save_images(cnt, fixed_noise)
        cnt += 1
        print("-----------------------------------------------------------------")
        #print(f"{epoch} epoch, Discriminator accuracy: {100 * discriminator_metric[1]}, Generator accuracy: {100 * generator_metric[1]}")
        print("Average Disc_fake loss: %f" % (np.mean(avg_disc_fake_loss)))
        print("Average Disc_real loss: %f" % (np.mean(avg_disc_real_loss)))
        print("Average GAN loss: %f" % (np.mean(avg_GAN_loss)))
        print("-----------------------------------------------------------------")
        discriminator.trainable = False
        generator.trainable = False
        # predict on fixed_noise
        generator.save(SAVE_MODEL_DIR+str(cnt)+"_TRIP_GEN.hdf5")
        discriminator.save(SAVE_MODEL_DIR+str(cnt)+"_TRIP_DISC.hdf5")
        if epoch > 1:
          disc_real_loss = np.array(avg_disc_real_loss)
          disc_fake_loss = np.array(avg_disc_fake_loss)
          GAN_loss = np.array(avg_GAN_loss)
          # Plot the losses vs training steps
          plt.figure(figsize=(16,8))
          plt.plot(range(0,epoch+1), disc_real_loss, label="Discriminator Loss - Real")
          plt.plot(range(0,epoch+1), disc_fake_loss, label="Discriminator Loss - Fake")
          plt.plot(range(0,epoch+1), GAN_loss, label="Generator Loss")
          plt.xlabel('Steps')
          plt.ylabel('Loss')
          plt.legend()
          plt.title('GAN Loss')
          plt.grid(True)
          plt.savefig('logs/losses_fig.png') 
          plt.show()




/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:81: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("se...)`


Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 128, 128, 3)       0         
_________________________________________________________________
sequential_1 (Sequential)    (None, 1)                 2804449   
Total params: 2,804,449
Trainable params: 2,802,529
Non-trainable params: 1,920
_________________________________________________________________
Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_2 (Dense)              (None, 32768)             3309568   
_________________________________________________________________
reshape_1 (Reshape)          (None, 8, 8, 512)         0         
_________________________________________________________________
up_sampling2d_1 (UpSampling2 (None, 16, 16, 512)       0         
_______________________

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:115: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("se...)`


Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 100)               0         
_________________________________________________________________
model_2 (Model)              (None, 128, 128, 3)       10135939  
_________________________________________________________________
model_1 (Model)              (None, 1)                 2804449   
Total params: 12,940,388
Trainable params: 10,134,147
Non-trainable params: 2,806,241
_________________________________________________________________
EPOCH: 0


KeyboardInterrupt: ignored